In [ ]:
import os

import rawutil as ru
from analytics.utils import get_car_query
from database.wintap_duckdb import WintapDuckDB, WintapDuckDBOptions

# initially, this will only work with a configured .env file in the top level directory.
data_path=os.getenv('DATAPATH')

# Initialize an in-memory db. Save reference in a variable and then set magic-duckdb environment. 
# Result is ability to use the same DB instance from python code and %dql/%%dql magics.
# Also create views for every top-level type found in the current dataset.
con=ru.init_db(data_path)
options = WintapDuckDBOptions(con, data_path)
wintap_duckdb = WintapDuckDB(options)

In [ ]:
# example query running single MITRE CAR analytic
car_query = get_car_query('car-2013-02-003')
print(wintap_duckdb.query(car_query))